THIS IS TO EXTRACT THE DATA FROM .mat File  ->  .csv file  

In [1]:
import os
import scipy.io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

THIS FUNCTION EXTRACTS THE FILES AND CONVERST THEM FROM "SPECIFIED FOLDER" : SPECIFIYING THE FOLDERS IS DONE MANUAALLY

In [2]:

def extract_and_save_mat_files(folder_path, columns_to_extract):
    csv_file_paths = []  # Store paths of created CSV files for later processing
    for file in os.listdir(folder_path):
        if file.endswith('.mat'):
            file_path = os.path.join(folder_path, file)
            mat_data = scipy.io.loadmat(file_path, squeeze_me=True, struct_as_record=False)
            dataset = mat_data.get('Dataset')
            if not dataset:
                print(f"No 'Dataset' found in {file}. Skipping.")
                continue

            data = {}
            array_lengths = {}

            for field in columns_to_extract:
                field_value = getattr(dataset, field, None)
                if field_value is not None:
                    if isinstance(field_value, np.ndarray):
                        data[field] = field_value.squeeze()
                        array_lengths[field] = len(data[field])
                    else:
                        data[field] = field_value
                        array_lengths[field] = 1

            if len(set(array_lengths.values())) == 1:
                df = pd.DataFrame(data)
                csv_file_name = 'New_' + os.path.splitext(file)[0] + '.csv'
                csv_file_path = os.path.join(folder_path, csv_file_name)
                df.to_csv(csv_file_path, index=False)
                csv_file_paths.append(csv_file_path)
                print(f"Processed and saved: {csv_file_name}")
            else:
                print(f"Skipping file '{file}' due to columns having different lengths.")
    return csv_file_paths


saves the CSV file location and changes the TIME -> rectifies the reset values in all

In [3]:

def correct_time_column_in_csv(csv_file_path):
    df = pd.read_csv(csv_file_path)
    # Assuming 'Time' is in a format that needs conversion
    df['Time'] = pd.to_timedelta(df['Time'])
    df['Time Hours'] = df['Time'].dt.total_seconds() / 3600
    resets = df['Time Hours'].diff() < 0
    df['Days Passed'] = resets.cumsum()
    df['Corrected Time Hours'] = df['Time Hours'] + df['Days Passed'] * 24

    # Save the corrected DataFrame back to CSV
    corrected_csv_file_path = csv_file_path.replace('.csv', '_corrected.csv')
    df.to_csv(corrected_csv_file_path, index=False)
    print(f"Corrected and saved: {corrected_csv_file_path}")


SPECIFY THE FOLDER TO BE EXTRACTED ALSO CHANGE THE LIST TO EXTRACT THE SPECIFIC COLUMNS 

In [4]:
def main():             
    '''# Define the columns you want to extract from cyclic ageging  
    columns_to_extract = [
        'Time', 'DataSet', 'tStep', 'Line', 'Command', 'U', 'I', 'Ah', 'AhStep', 'AhSet',
        'AhChSet', 'AhDisSet', 'Wh', 'WhStep', 'T1', 'RAC', 'RDC', 'CycCount', 'Count',
        'State', 'i_cut_hi', 'i_cut_lo', 'i_cut_low', 'i_cut_high'
    ]'''

    # Define the columns you want to extract from checkup cyclic aging
    columns_to_extract = [
        'Time', 'DataSet', 'tStep', 'Line', 'Command', 'U', 'I', 'Ah', 'AhStep', 'AhSet',
         'Wh',  'T1', 'RAC', 'RDC', 'CycCount', 
        'State']
    folder_path = 'E:\Thesis CEVT\Dataset\TUM\CU_Cyclic'
    for subdir , dirs ,files in os.walk(folder_path, topdown=True):
        for folder_name in dirs:
            if folder_name.endswith("cyc"):
                folder_path=os.path.join(subdir,folder_name)
                extract_and_save_mat_files(folder_path,columns_to_extract)

    


In [5]:

if __name__ == "__main__":
    main()


Processed and saved: New_BW-VTC-210_2228_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-211_2229_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-217_2230_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-218_2231_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-219_2232_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-221_2485_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-222_2507_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-223_2486_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-230_2508_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-231_2509_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-232_2487_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-233_2488_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-234_2489_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-235_1079_CU_cyc_000_BW-VTC-CYC.csv
Processed and saved: New_BW-VTC-236_1151_CU_cyc_000_BW-VTC-CYC

KeyboardInterrupt: 